In [2]:
# !pip install yfpy

In [125]:
import pandas as pd
import requests
from dotenv import load_dotenv
import os
from yfpy.query import YahooFantasySportsQuery
from yfpy.models import Scoreboard, Matchup, YahooFantasyObject
from yahoo_oauth import OAuth2
import xml.etree.ElementTree as ET
from collections import defaultdict


In [4]:
load_dotenv()
CLIENT_ID = os.getenv("CLIENT_ID")
CLIENT_SECRET = os.getenv("CLIENT_SECRET")

In [5]:
def strip_namespace(tag):
    """Remove namespace from the tag name."""
    return tag.split("}")[-1] if "}" in tag else tag

def parse_element(element):
    parsed_data = {}
    children = list(element)

    # Check if all children have the same tag (implying a list)
    child_tags = [strip_namespace(child.tag) for child in children]
    is_list = len(set(child_tags)) == 1 and len(children) > 1

    # Convert child elements into dictionary keys
    if is_list:
        return [parse_element(child) for child in children]  # Return a list directly
    else:
        for child in children:
            tag = strip_namespace(child.tag)
            child_data = parse_element(child)

            # Handle multiple children with the same tag by storing them as lists
            if tag in parsed_data:
                if not isinstance(parsed_data[tag], list):
                    parsed_data[tag] = [parsed_data[tag]]
                parsed_data[tag].append(child_data)
            else:
                parsed_data[tag] = child_data

    # If the element has text content, add it
    text = element.text.strip() if element.text else ""
    if text and not parsed_data:
        return text  # Return text if no nested structure

    return parsed_data or text  # Return text if no nested structure

def xml_to_dict(xml_string):
    root = ET.fromstring(xml_string)
    return {strip_namespace(root.tag): parse_element(root)}

In [6]:
# Use last season stats so there aren't any spoilers
current = False
league_id = "67269" if current else "97108"
game_id = 453 if current else 427

query = YahooFantasySportsQuery(
    league_id=league_id,
    game_code="nhl",
    game_id=game_id,
    yahoo_consumer_key=CLIENT_ID,
    yahoo_consumer_secret=CLIENT_SECRET
)

oauth = OAuth2(
    CLIENT_ID,
    CLIENT_SECRET,
    browser_callback=True,
)

In [7]:
BASE_URL = "https://fantasysports.yahooapis.com/fantasy/v2"

In [8]:
def api_request(url):
    if not oauth.token_is_valid():
        oauth.refresh_access_token()
    headers = {
        "Authorization": f"Bearer {oauth.access_token}",
        "Content-Type": "application/json"
    }
    response = requests.get(url, headers=headers)
    if response.status_code != 200:
        print(response.text)
        response.raise_for_status()
    xml = response.text
    data = xml_to_dict(response.text)["fantasy_content"]
    return data

In [9]:
def get_league():
    league_key = f"{game_id}.l.{league_id}"
    url = f"{BASE_URL}/league/{league_key};out=standings"
    resp_data = api_request(url)["league"]
    return resp_data

In [10]:
league = get_league()
teams = league["standings"]["teams"]
teams

[{'team_key': '427.l.97108.t.2',
  'team_id': '2',
  'name': 'Eric’s a Bitch Team',
  'url': 'https://hockey.fantasysports.yahoo.com/2023/hockey/97108/2',
  'team_logos': {'team_logo': {'size': 'large',
    'url': 'https://s.yimg.com/cv/apiv2/default/nhl/nhl_12_k.png'}},
  'waiver_priority': '4',
  'number_of_moves': '59',
  'number_of_trades': '0',
  'roster_adds': {'coverage_type': 'week',
   'coverage_value': '26',
   'value': '0'},
  'clinched_playoffs': '1',
  'league_scoring_type': 'headpoint',
  'draft_position': '4',
  'has_draft_grade': '0',
  'managers': {'manager': {'manager_id': '2',
    'nickname': 'Ken',
    'guid': 'TPMH3J2R4LWWFJV4GPPLVJPZYI',
    'email': 'dragonkenkj@gmail.com',
    'image_url': 'https://s.yimg.com/ag/images/default_user_profile_pic_64sq.jpg',
    'felo_score': '656',
    'felo_tier': 'silver'}},
  'team_stats': {'coverage_type': 'season',
   'season': '2023',
   'stats': [{'stat_id': '1', 'value': '304'},
    {'stat_id': '2', 'value': '478'},
    {'s

In [11]:
def get_team_name(team_key):
    team_name = next((team["name"] for team in teams if team["team_key"] == team_key), 'Unknown Team')
    return team_name 

In [12]:
LEAGUE_KEY = query.get_league_key(season=None)

In [13]:
league_info = query.get_league_info()

In [14]:
def get_players(player_keys):
    return [
        player 
        for i in range(int(len(player_keys) / 25) + 1)
        for player in query.query(
            f"https://fantasysports.yahooapis.com/fantasy/v2/league/{league_info.league_key}/players;player_keys={','.join(player_keys[i*25:min((i+1)*25, len(player_keys))])};start={i*25}/stats",
            ["league", "players"]
        )
    ]

In [15]:
team_info = query.get_league_teams()
team_info[3]

Team({
  "draft_position": 2,
  "has_draft_grade": 0,
  "league_scoring_type": "headpoint",
  "managers": {
    "manager": {
      "email": "andrewp3@shaw.ca",
      "felo_score": 530,
      "felo_tier": "bronze",
      "guid": "OUL766QQDYAPQNVXMSEDVLET24",
      "image_url": "https://s.yimg.com/ag/images/8bb55647-c4ee-4c08-9bc5-fa5609b44ae3_64sq.jpg",
      "manager_id": 4,
      "nickname": "Andrew"
    }
  },
  "name": "Miami Steamrollers",
  "number_of_moves": 10,
  "number_of_trades": 0,
  "roster_adds": {
    "coverage_type": "week",
    "coverage_value": 26,
    "value": 0
  },
  "team_id": 4,
  "team_key": "427.l.97108.t.4",
  "team_logos": {
    "team_logo": {
      "size": "large",
      "url": "https://yahoofantasysports-res.cloudinary.com/image/upload/t_s192sq/fantasy-logos/487f5795c5a24fc01ad380d9da51fb26e062fd152c0060b938512522089524b1.png"
    }
  },
  "url": "https://hockey.fantasysports.yahoo.com/2023/hockey/97108/4",
  "waiver_priority": 6
})

In [16]:
def get_full_scoring_df():
    league_settings = query.get_league_settings()
    scoring_categories = league_settings.stat_categories.stats
    for i in range(len(scoring_categories)):
        scoring_categories[i] = scoring_categories[i].clean_data_dict()
    scoring_categories = pd.DataFrame(data=scoring_categories)

    scoring_modifiers = league_settings.stat_modifiers.stats
    scoring_modifiers = [scoring_modifiers[i].clean_data_dict() for i in range(len(scoring_modifiers))]

    scoring_settings = pd.merge(left=scoring_categories, right=pd.DataFrame(data=scoring_modifiers), on='stat_id', how='inner')
    return scoring_settings


def get_scoring_dict_by_stat_id(scoring_settings_df):
    return {str(scoring_settings_df['stat_id'][i]) : float(scoring_settings_df['value'][i]) for i in range(len(scoring_settings_df))}

In [17]:
# LEAGUE_SETTINGS = query.get_league_settings()
# SCORING_CATEGORIES = LEAGUE_SETTINGS.stat_categories.stats
# for i in range(len(SCORING_CATEGORIES)):
#     SCORING_CATEGORIES[i] = SCORING_CATEGORIES[i].clean_data_dict()
# SCORING_CATEGORIES = pd.DataFrame(data=SCORING_CATEGORIES)

# SCORING_MODIFIERS = LEAGUE_SETTINGS.stat_modifiers.stats
# SCORING_MODIFIERS = [SCORING_MODIFIERS[i].clean_data_dict() for i in range(len(SCORING_MODIFIERS))]

# SCORING_SETTINGS = pd.merge(left=SCORING_CATEGORIES, right=pd.DataFrame(data=SCORING_MODIFIERS), on='stat_id', how='inner')

In [18]:
SCORING_SETTINGS = get_full_scoring_df()
SCORING_MODIFIERS_BY_STAT_ID = get_scoring_dict_by_stat_id(SCORING_SETTINGS)

Draft Info

In [19]:
full_draft = query.get_league_draft_results()

Seperate Draft into by Team

In [20]:
full_draft = pd.DataFrame(data=full_draft, columns=['Overall', 'Round', 'Team ID', 'Player ID'])

team_drafts = {}
team_keys = []
for ID in range(1, full_draft['Team ID'].nunique()+1):
    team_keys = [LEAGUE_KEY + '.t.' + str(ID) for ID in range(1, full_draft['Team ID'].nunique()+1)] 
team_keys = tuple(team_keys)

In [21]:
team_drafts = {team_key : full_draft[full_draft['Team ID'] == team_key] for team_key in team_keys}

Assign Best and Worst Draft

In [39]:
'''
Best/Worst Draft
'''
total_draft_points = dict.fromkeys(team_keys, 0)
for team in team_drafts:
    # pull each team draft 
    draft = list(team_drafts[team]['Player ID'])
    stats = get_players(player_keys=draft)
    
    for j in range(len(draft)):
        # pull each player from the draft
        # pull each player's stats
        player_stats = stats[j].player_stats.clean_data_dict()['stats']
        
        for i in range(len(player_stats)):
            # pull league scoring settings
            # multiply league settings by player's stats
            stat = player_stats[i]['stat'].clean_data_dict()
            if str(stat['stat_id']) in list(SCORING_MODIFIERS_BY_STAT_ID.keys()):
                total_draft_points[team] += stat['value'] * SCORING_MODIFIERS_BY_STAT_ID[str(stat['stat_id'])] #list(SCORING_SETTINGS['value'])[list(SCORING_SETTINGS['stat_id']).index(stat['stat_id'])]

total_draft_points = {key : round(value, 1) for key, value in total_draft_points.items()}
total_draft_points = dict(sorted(total_draft_points.items(), key=lambda item: item[1], reverse=True))
total_draft_points = {key : (get_team_name(key), value) for key, value in total_draft_points.items()}

top_x = 4
print('The best drafts are...')
for i in range(top_x):
    print(f'{i+1}. {list(total_draft_points.values())[i][0]} ({str(list(total_draft_points.values())[i][1])})')
print('')

print('The worst drafts are...')
for i in range(top_x):
    print(f'{i+1}. {list(total_draft_points.values())[len(total_draft_points)-i-1][0]} ({str(list(total_draft_points.values())[len(total_draft_points)-1][1])})')

The best drafts are...
1. Tim's Terrific Team (5554.2)
2. Theodore's Poo Poo Hospital (5315.4)
3. Eric’s a Bitch Team (5312.0)
4. eric's Awe-Inspiring Team (5007.2)

The worst drafts are...
1. Miami Steamrollers (4597.7)
2. Kevin's Incredible Team (4597.7)
3. Cyrus's Cursed Team (4597.7)
4. adam2 (4597.7)


Mike Sillinger Award - Most Picked up/Dropped Player

In [40]:
'''
Mike Sillinger Award
Evaluated by who was dropped the most
Prints Top 10
'''
transactions = query.get_league_transactions()

drops = {}
missed = 0
for transaction in transactions:
    if transaction.type == 'drop' and transaction.status == 'successful':
        try:
            drops[transaction.players[0].player_key][1] += 1
        except:
            drops[transaction.players[0].player_key] = [transaction.players[0].full_name, 1]
    elif transaction.type == 'add/drop' and transaction.status == 'successful':
        try:
            drops[transaction.players[1].player_key][1] += 1
        except:
            drops[transaction.players[1].player_key] = [transaction.players[1].full_name, 1]

drops = dict(sorted(drops.items(), key=lambda item: item[1][1], reverse=True))

top_x = 20
print('The Most Picked-Up/Dropped Players are...')
for i in range(top_x):
    print(f'{i+1}. {list(drops.values())[i][0]} ({str(list(drops.values())[i][1])})')

The Most Picked-Up/Dropped Players are...
1. Matt Boldy (7)
2. Ukko-Pekka Luukkonen (6)
3. Darnell Nurse (5)
4. Jakob Chychrun (5)
5. Jacob Markstrom (5)
6. Brent Burns (5)
7. Claude Giroux (5)
8. Tyler Toffoli (4)
9. Ilya Samsonov (4)
10. Drake Batherson (4)
11. Josh Manson (4)
12. Connor Bedard (4)
13. Logan Thompson (4)
14. Jeff Skinner (4)
15. Tom Wilson (3)
16. Filip Gustavsson (3)
17. Joey Daccord (3)
18. Boone Jenner (3)
19. Alex Lyon (3)
20. Joseph Woll (3)


Closest Weekly Matchups, Biggest Blowouts, and Lopsided Rivalry

In [24]:
def get_league_matchup_results_by_week(weeks: list[int]):
    weeks = ','.join(str(week) for week in weeks)
    url = f"{BASE_URL}/league/{LEAGUE_KEY}/scoreboard;week={weeks}"
    results = api_request(url=url)
    return results['league']['scoreboard']['matchups']

In [119]:
'''
Gets a dictionary with completed weeks matchup information
Used in Closed Matchups, Biggest Blowouts, and Lopsided Rivalry Awards
'''

start_week = int(league['start_week'])
end_week = int(league['end_week'])
weeks = list(range(start_week, end_week+1))
weekly_matchup_data = get_league_matchup_results_by_week(weeks)
# matchup_num = 0 to len(weekly_matchup_data)-1
# team_num = 0 or 1
# weekly_matchup_data[matchup_num]['teams'][team_num]['team_points']['total']

completed_matchups_data = []
for matchup_num in range(len(weekly_matchup_data)):
    completed_matchups_data.append({
        'team1_key' : weekly_matchup_data[matchup_num]['teams'][0]['team_key'],
        'team1_name' : weekly_matchup_data[matchup_num]['teams'][0]['name'],
        'team1_points' : float(weekly_matchup_data[matchup_num]['teams'][0]['team_points']['total']),
        'team2_key' : weekly_matchup_data[matchup_num]['teams'][1]['team_key'],
        'team2_name' : weekly_matchup_data[matchup_num]['teams'][1]['name'],
        'team2_points' : float(weekly_matchup_data[matchup_num]['teams'][1]['team_points']['total']),
        'point_diff' : round(float(weekly_matchup_data[matchup_num]['teams'][0]['team_points']['total']) - float(weekly_matchup_data[matchup_num]['teams'][1]['team_points']['total']), 2),
        'is_tied' : int(weekly_matchup_data[matchup_num]['is_tied']),
        'week' : int(weekly_matchup_data[matchup_num]['week']),
        'is_playoffs' : int(weekly_matchup_data[matchup_num]['is_playoffs']),
        'is_consolation' : int(weekly_matchup_data[matchup_num]['is_consolation'])
    })

sorted_matchups_data = sorted(completed_matchups_data, key=lambda x: abs(x['point_diff']))

In [181]:
'''
Closest Matchups
'''
top_x = 10
print('The top barn burners were...')
narrow_wins = {team_key : 0 for team_key in team_keys}
narrow_losses = {team_key : 0 for team_key in team_keys}
narrow_matchups = {team_key : 0 for team_key in team_keys}
for i in range(top_x):
    if sorted_matchups_data[i]['is_tied']:
        print(f"{i+1}. {sorted_matchups_data[i]['team1_name']} and {sorted_matchups_data[i]['team2_name']} tied at {sorted_matchups_data[i]['team1_points']}")
    elif sorted_matchups_data[i]['point_diff'] > 0:
        print(f"{i+1}. {sorted_matchups_data[i]['team1_name']} def. {sorted_matchups_data[i]['team2_name']}")
        print(f"Score: {sorted_matchups_data[i]['team1_points']}-{sorted_matchups_data[i]['team2_points']}") 
        print(f"Point Differential: {abs(sorted_matchups_data[i]['point_diff'])}")
        print(f"Week: {sorted_matchups_data[i]['week']}")
        narrow_wins[sorted_matchups_data[i]['team1_key']] += 1
        narrow_losses[sorted_matchups_data[i]['team2_key']] += 1
        narrow_matchups[sorted_matchups_data[i]['team1_key']] += 1
        narrow_matchups[sorted_matchups_data[i]['team2_key']] += 1
    elif sorted_matchups_data[i]['point_diff'] < 0:
        print(f"{i+1}. {sorted_matchups_data[i]['team2_name']} def. {sorted_matchups_data[i]['team1_name']}")
        print(f"Score: {sorted_matchups_data[i]['team2_points']}-{sorted_matchups_data[i]['team1_points']}") 
        print(f"Point Differential: {abs(sorted_matchups_data[i]['point_diff'])}")
        print(f"Week: {sorted_matchups_data[i]['week']}")
        narrow_wins[sorted_matchups_data[i]['team2_key']] += 1
        narrow_losses[sorted_matchups_data[i]['team1_key']] += 1
        narrow_matchups[sorted_matchups_data[i]['team1_key']] += 1
        narrow_matchups[sorted_matchups_data[i]['team2_key']] += 1
    print('')

# This can be done better
# It currently just takes who won the most, who lost the most, and who appeared the most in the top 10 (and doesn't deal with ties of these, just shows 1 of them)
# But this can be turned into who has the best record in matchups decided by less then x (5-10 probably) points
# Can also see who has the most matchups decided by less then x points
# i.e. a different award (Best/Worst Close Matchup Record)
# Award for loser of most blowouts too (50+ pts)? - Might not be that interesting will likely just be people near the bottom of the table
narrow_wins = sorted(narrow_wins.items(), key=lambda item: item[1], reverse=True)
narrow_losses = sorted(narrow_losses.items(), key=lambda item: item[1], reverse=True)
narrow_matchups = sorted(narrow_matchups.items(), key=lambda item: item[1], reverse=True)
print(f"Wow! {get_team_name(narrow_wins[0][0])} you love winning by the skin of your teeth, you won {narrow_wins[0][1]} close matchups!")
print(f"Unlucky {get_team_name(narrow_losses[0][0])}, you lost {narrow_losses[0][1]} close matchups :(")
print(f"{get_team_name(narrow_matchups[0][0])}, you have nerves of steel. You were in {narrow_matchups[0][1]} close matchups!")

The top barn burners were...
1. Theodore's Poo Poo Hospital def. Miami Steamrollers
Score: 177.1-176.7
Point Differential: 0.4
Week: 6

2. Tim's Terrific Team def. Cyrus's Cursed Team
Score: 213.2-212.7
Point Differential: 0.5
Week: 8

3. Kevin's Incredible Team def. Miami Steamrollers
Score: 191.5-191.0
Point Differential: 0.5
Week: 9

4. Tim's Terrific Team def. Eric’s a Bitch Team
Score: 185.6-184.6
Point Differential: 1.0
Week: 6

5. Kevin's Incredible Team def. Tim's Terrific Team
Score: 167.9-166.2
Point Differential: 1.7
Week: 3

6. Eric’s a Bitch Team def. eric's Awe-Inspiring Team
Score: 191.0-188.8
Point Differential: 2.2
Week: 12

7. adam2 def. Kevin's Incredible Team
Score: 158.6-156.1
Point Differential: 2.5
Week: 8

8. Cyrus's Cursed Team def. Eric’s a Bitch Team
Score: 173.5-170.9
Point Differential: 2.6
Week: 3

9. Eric’s a Bitch Team def. Theodore's Poo Poo Hospital
Score: 212.2-209.0
Point Differential: 3.2
Week: 23

10. Kevin's Incredible Team def. adam2
Score: 165.2

In [ ]:
'''
Biggest Blowouts (Boat Raced Award)
'''
top_x = 10
num = 1
print('These teams got BOATRACED these weeks')
print('The biggest blowouts were...')
for i in range(len(sorted_matchups_data)-1, len(sorted_matchups_data)-top_x-1, -1):
    if sorted_matchups_data[i]['point_diff'] > 0:
        print(f"{num}. {sorted_matchups_data[i]['team1_name']} def. {sorted_matchups_data[i]['team2_name']}")
        print(f"Score: {sorted_matchups_data[i]['team1_points']}-{sorted_matchups_data[i]['team2_points']}") 
        print(f"Point Differential: {abs(sorted_matchups_data[i]['point_diff'])}")
        print(f"Week: {sorted_matchups_data[i]['week']}")
    elif sorted_matchups_data[i]['point_diff'] < 0:
        print(f"{num}. {sorted_matchups_data[i]['team2_name']} def. {sorted_matchups_data[i]['team1_name']}")
        print(f"Score: {sorted_matchups_data[i]['team2_points']}-{sorted_matchups_data[i]['team1_points']}") 
        print(f"Point Differential: {abs(sorted_matchups_data[i]['point_diff'])}")
        print(f"Week: {sorted_matchups_data[i]['week']}")
    print('')
    num += 1



You got BOATRACED
The biggest blowouts were...
1. Theodore's Poo Poo Hospital def. Kevin's Incredible Team
Score: 262.8-124.3
Point Differential: 138.5
Week: 18

2. Eric’s a Bitch Team def. Miami Steamrollers
Score: 243.5-123.4
Point Differential: 120.1
Week: 11

3. Cyrus's Cursed Team def. Miami Steamrollers
Score: 200.0-108.4
Point Differential: 91.6
Week: 14

4. eric's Awe-Inspiring Team def. adam2
Score: 209.6-118.0
Point Differential: 91.6
Week: 14

5. Tim's Terrific Team def. Miami Steamrollers
Score: 198.0-106.6
Point Differential: 91.4
Week: 5

6. Theodore's Poo Poo Hospital def. Miami Steamrollers
Score: 199.0-108.0
Point Differential: 91.0
Week: 13

7. Cyrus's Cursed Team def. Miami Steamrollers
Score: 192.6-106.8
Point Differential: 85.8
Week: 21

8. Cyrus's Cursed Team def. Kevin's Incredible Team
Score: 250.3-165.1
Point Differential: 85.2
Week: 5

9. Theodore's Poo Poo Hospital def. adam2
Score: 276.7-192.5
Point Differential: 84.2
Week: 19

10. Tim's Terrific Team def. K

In [ ]:
'''
Team Dominance Over Another Team (Lopsided Rivalry, Pure Dominance)

- Seems as if the team keys are always ordered the same with the smallest team ID first
  But if this assumption is ever not correct it will completely mess up the stat
  Could add error handling for this scenario but may not be necessary

- incl_playoffs and incl_consolation tags could make for some fun statements on the frontend 
  but this does not handle if a team played in multiple playoff games with some being non-consolation and others being consolation
  (but I guess you would only play a team once in the playoffs so if they played playoffs and non-conslation it guarantees there was only 1 matchup and it was real playoffs) - I think this statement is TRUE
  (not sure if teams can play multiple times in consolation, sometimes consolation could be a ladder format where you can go up and down ex. ESPN fantasy)
'''
matchup_dict = defaultdict(list)
for matchup in completed_matchups_data:
    key = frozenset([matchup['team1_key'], matchup['team2_key']])
    matchup_dict[key].append(matchup)
matchup_dict = dict(matchup_dict)

season_matchup_stats = {}
for matchup_combination in matchup_dict.keys():
    team1_total_points = 0
    team2_total_points = 0
    for matchup in matchup_dict[matchup_combination]:
        team1_total_points += matchup['team1_points']
        team2_total_points += matchup['team2_points']
        season_matchup_stats[matchup_combination] = {
            'team1_key' : matchup['team1_key'],
            'team1_name' : matchup['team1_name'],
            'team1_total_points' : round(team1_total_points, 2),
            'team2_key' : matchup['team2_key'],
            'team2_name' : matchup['team2_name'],
            'team2_total_points' : round(team2_total_points, 2),
            'incl_playoffs' : matchup['is_playoffs'],
            'incl_consolation' :matchup['is_consolation']
        }
    season_matchup_stats[matchup_combination]['total_point_diff'] = round(season_matchup_stats[matchup_combination]['team1_total_points'] - season_matchup_stats[matchup_combination]['team2_total_points'], 2)

sorted_season_matchup_stats = sorted(season_matchup_stats.items(), key=lambda matchup: abs(matchup[1]['total_point_diff']), reverse=True)

top_x = 10
print('Some matchups were never fair from the start')
print('The most lopsided season matchups were...')
for i in range(top_x):
    if sorted_season_matchup_stats[i][1]['total_point_diff'] > 0:
        print(f"{i+1}. {sorted_season_matchup_stats[i][1]['team1_name']} def. {sorted_season_matchup_stats[i][1]['team2_name']}")
        print(f"Total Score: {sorted_season_matchup_stats[i][1]['team1_total_points']}-{sorted_season_matchup_stats[i][1]['team2_total_points']}") 
        print(f"Total Point Differential: {abs(sorted_season_matchup_stats[i][1]['total_point_diff'])}")
        print(f"Includes Playoffs: {'Yes' if sorted_season_matchup_stats[i][1]['incl_playoffs'] else 'No'}")
        print(f"Includes Consolation: {'Yes' if sorted_season_matchup_stats[i][1]['incl_consolation'] else 'No'}")
    elif sorted_season_matchup_stats[i][1]['total_point_diff'] < 0:
        print(f"{i+1}. {sorted_season_matchup_stats[i][1]['team2_name']} def. {sorted_season_matchup_stats[i][1]['team1_name']}")
        print(f"Total Score: {sorted_season_matchup_stats[i][1]['team2_total_points']}-{sorted_season_matchup_stats[i][1]['team1_total_points']}") 
        print(f"Total Point Differential: {abs(sorted_season_matchup_stats[i][1]['total_point_diff'])}")
        print(f"Includes Playoffs: {'Yes' if sorted_season_matchup_stats[i][1]['incl_playoffs'] else 'No'}")
        print(f"Includes Consolation: {'Yes' if sorted_season_matchup_stats[i][1]['incl_consolation'] else 'No'}")
    print('')

Some matchups were never fair from the start
The most lopsided season matchups were...
1. Eric’s a Bitch Team def. Miami Steamrollers
Total Score: 592.3-381.6
Total Point Differential: 210.7
Includes Playoffs: No
Includes Consolation: No

2. Theodore's Poo Poo Hospital def. Kevin's Incredible Team
Total Score: 623.7-414.7
Total Point Differential: 209.0
Includes Playoffs: No
Includes Consolation: No

3. Tim's Terrific Team def. Miami Steamrollers
Total Score: 587.5-406.7
Total Point Differential: 180.8
Includes Playoffs: No
Includes Consolation: No

4. Cyrus's Cursed Team def. Kevin's Incredible Team
Total Score: 660.3-480.0
Total Point Differential: 180.3
Includes Playoffs: No
Includes Consolation: No

5. eric's Awe-Inspiring Team def. adam2
Total Score: 826.3-659.9
Total Point Differential: 166.4
Includes Playoffs: Yes
Includes Consolation: No

6. Theodore's Poo Poo Hospital def. Miami Steamrollers
Total Score: 588.3-422.1
Total Point Differential: 166.2
Includes Playoffs: No
Include

In [172]:
sorted_season_matchup_stats

[(frozenset({'427.l.97108.t.2', '427.l.97108.t.4'}),
  {'team1_key': '427.l.97108.t.2',
   'team1_name': 'Eric’s a Bitch Team',
   'team1_total_points': 592.3,
   'team2_key': '427.l.97108.t.4',
   'team2_name': 'Miami Steamrollers',
   'team2_total_points': 381.6,
   'total_point_diff': 210.7}),
 (frozenset({'427.l.97108.t.3', '427.l.97108.t.8'}),
  {'team1_key': '427.l.97108.t.3',
   'team1_name': "Theodore's Poo Poo Hospital",
   'team1_total_points': 623.7,
   'team2_key': '427.l.97108.t.8',
   'team2_name': "Kevin's Incredible Team",
   'team2_total_points': 414.7,
   'total_point_diff': 209.0}),
 (frozenset({'427.l.97108.t.4', '427.l.97108.t.6'}),
  {'team1_key': '427.l.97108.t.4',
   'team1_name': 'Miami Steamrollers',
   'team1_total_points': 406.7,
   'team2_key': '427.l.97108.t.6',
   'team2_name': "Tim's Terrific Team",
   'team2_total_points': 587.5,
   'total_point_diff': -180.8}),
 (frozenset({'427.l.97108.t.7', '427.l.97108.t.8'}),
  {'team1_key': '427.l.97108.t.7',
   '

Playground

In [29]:
drops.values()

dict_values([['Matt Boldy', 7], ['Ukko-Pekka Luukkonen', 6], ['Darnell Nurse', 5], ['Jakob Chychrun', 5], ['Jacob Markstrom', 5], ['Brent Burns', 5], ['Claude Giroux', 5], ['Tyler Toffoli', 4], ['Ilya Samsonov', 4], ['Drake Batherson', 4], ['Josh Manson', 4], ['Connor Bedard', 4], ['Logan Thompson', 4], ['Jeff Skinner', 4], ['Tom Wilson', 3], ['Filip Gustavsson', 3], ['Joey Daccord', 3], ['Boone Jenner', 3], ['Alex Lyon', 3], ['Joseph Woll', 3], ['Antti Raanta', 3], ['Jared McCann', 3], ['Elias Lindholm', 3], ['Laurent Brossoit', 3], ['Adam Larsson', 3], ['Rickard Rakell', 3], ['Brayden McNabb', 3], ['Mattias Ekholm', 3], ['William Karlsson', 3], ['Jonathan Quick', 3], ['Carter Hart', 3], ['Owen Tippett', 3], ['Bowen Byram', 2], ['Jake McCabe', 2], ['Thatcher Demko', 2], ['Scott Wedgewood', 2], ['Mark Scheifele', 2], ['Jonas Johansson', 2], ['Jeremy Swayman', 2], ['Gustav Forsling', 2], ['Karel Vejmelka', 2], ['David Rittich', 2], ['Neal Pionk', 2], ['Seth Jarvis', 2], ['Frederik Ander

In [30]:
transaction_type = []
for transaction in transactions:
    transaction_type.append(transaction.type)

set(transaction_type)

{'add', 'add/drop', 'commish', 'drop', 'trade'}

In [31]:
weeks = [1]
','.join(str(week) for week in weeks)

'1'

In [32]:
weeks = [1, 2, 3, 4, 5, 6]
weeks = ','.join(str(week) for week in weeks)
url = f"https://fantasysports.yahooapis.com/fantasy/v2/league/{LEAGUE_KEY}/scoreboard;week={weeks}"
week1 = api_request(url=url)
week1

{'league': {'league_key': '427.l.97108',
  'league_id': '97108',
  'name': 'Men2',
  'url': 'https://hockey.fantasysports.yahoo.com/2023/hockey/97108',
  'logo_url': '',
  'password': '',
  'draft_status': 'postdraft',
  'num_teams': '8',
  'edit_key': '2024-03-31',
  'weekly_deadline': 'intraday',
  'league_update_timestamp': '1711958417',
  'scoring_type': 'headpoint',
  'league_type': 'private',
  'renew': '',
  'renewed': '453_67269',
  'felo_tier': 'bronze',
  'iris_group_chat_id': '',
  'short_invitation_url': 'https://hockey.fantasysports.yahoo.com/2023/hockey/97108/invitation?key=3ce97a4f81b53dce&ikey=3860fd06c80066cd',
  'allow_add_to_dl_extra_pos': '1',
  'is_pro_league': '0',
  'is_cash_league': '0',
  'current_week': '24',
  'start_week': '1',
  'start_date': '2023-10-10',
  'end_week': '24',
  'end_date': '2024-03-31',
  'is_finished': '1',
  'is_plus_league': '0',
  'game_code': 'nhl',
  'season': '2023',
  'scoreboard': {'week': '1,2,3,4,5,6',
   'matchups': [{'week': '1

In [33]:
week1['league']['scoreboard']['matchups'][3]['teams'][0]['team_points']['total']

'182.60'

In [34]:
week1['league']['scoreboard']['matchups'][3]

{'week': '1',
 'week_start': '2023-10-10',
 'week_end': '2023-10-15',
 'status': 'postevent',
 'is_playoffs': '0',
 'is_consolation': '0',
 'is_tied': '0',
 'winner_team_key': '427.l.97108.t.6',
 'stat_winners': [{'stat_id': '1', 'winner_team_key': '427.l.97108.t.6'},
  {'stat_id': '2', 'winner_team_key': '427.l.97108.t.7'},
  {'stat_id': '4', 'winner_team_key': '427.l.97108.t.6'},
  {'stat_id': '8', 'winner_team_key': '427.l.97108.t.7'},
  {'stat_id': '11', 'winner_team_key': '427.l.97108.t.7'},
  {'stat_id': '14', 'winner_team_key': '427.l.97108.t.7'},
  {'stat_id': '31', 'winner_team_key': '427.l.97108.t.6'},
  {'stat_id': '32', 'winner_team_key': '427.l.97108.t.6'},
  {'stat_id': '19', 'is_tied': '1'},
  {'stat_id': '22', 'is_tied': '1'},
  {'stat_id': '25', 'winner_team_key': '427.l.97108.t.6'},
  {'stat_id': '27', 'is_tied': '1'}],
 'teams': [{'team_key': '427.l.97108.t.6',
   'team_id': '6',
   'name': "Tim's Terrific Team",
   'url': 'https://hockey.fantasysports.yahoo.com/2023

In [35]:
week1

{'league': {'league_key': '427.l.97108',
  'league_id': '97108',
  'name': 'Men2',
  'url': 'https://hockey.fantasysports.yahoo.com/2023/hockey/97108',
  'logo_url': '',
  'password': '',
  'draft_status': 'postdraft',
  'num_teams': '8',
  'edit_key': '2024-03-31',
  'weekly_deadline': 'intraday',
  'league_update_timestamp': '1711958417',
  'scoring_type': 'headpoint',
  'league_type': 'private',
  'renew': '',
  'renewed': '453_67269',
  'felo_tier': 'bronze',
  'iris_group_chat_id': '',
  'short_invitation_url': 'https://hockey.fantasysports.yahoo.com/2023/hockey/97108/invitation?key=3ce97a4f81b53dce&ikey=3860fd06c80066cd',
  'allow_add_to_dl_extra_pos': '1',
  'is_pro_league': '0',
  'is_cash_league': '0',
  'current_week': '24',
  'start_week': '1',
  'start_date': '2023-10-10',
  'end_week': '24',
  'end_date': '2024-03-31',
  'is_finished': '1',
  'is_plus_league': '0',
  'game_code': 'nhl',
  'season': '2023',
  'scoreboard': {'week': '1,2,3,4,5,6',
   'matchups': [{'week': '1

In [36]:
scoreboard = Scoreboard(week1['league']['scoreboard'])

In [37]:
scoreboard.matchups()

KeyError: 'matchup'

In [ ]:
query_week1 = query.get_league_scoreboard_by_week(chosen_week=1)
query_week1

Scoreboard({
  "matchups": [
    {
      "matchup": {
        "is_consolation": 0,
        "is_playoffs": 0,
        "is_tied": 0,
        "status": "postevent",
        "teams": [
          {
            "team": {
              "clinched_playoffs": 1,
              "draft_position": 7,
              "has_draft_grade": 0,
              "is_owned_by_current_login": 1,
              "league_scoring_type": "headpoint",
              "managers": {
                "manager": {
                  "email": "amtadam14@hotmail.ca",
                  "felo_score": 577,
                  "felo_tier": "bronze",
                  "guid": "NGSXRXJK2U3UJVTCC2PO3L2A6M",
                  "image_url": "https://s.yimg.com/ag/images/default_user_profile_pic_64sq.jpg",
                  "is_commissioner": 0,
                  "is_current_login": 1,
                  "manager_id": 1,
                  "nickname": "adam"
                }
              },
              "name": "adam2",
              "number_

In [ ]:
query_week1.clean_data_dict()['matchups'][0]

{'matchup': Matchup({
   "is_consolation": 0,
   "is_playoffs": 0,
   "is_tied": 0,
   "status": "postevent",
   "teams": [
     {
       "team": {
         "clinched_playoffs": 1,
         "draft_position": 7,
         "has_draft_grade": 0,
         "is_owned_by_current_login": 1,
         "league_scoring_type": "headpoint",
         "managers": {
           "manager": {
             "email": "amtadam14@hotmail.ca",
             "felo_score": 577,
             "felo_tier": "bronze",
             "guid": "NGSXRXJK2U3UJVTCC2PO3L2A6M",
             "image_url": "https://s.yimg.com/ag/images/default_user_profile_pic_64sq.jpg",
             "is_commissioner": 0,
             "is_current_login": 1,
             "manager_id": 1,
             "nickname": "adam"
           }
         },
         "name": "adam2",
         "number_of_moves": 23,
         "number_of_trades": 0,
         "roster_adds": {
           "coverage_type": "week",
           "coverage_value": 26,
           "value": 0
   

In [ ]:
query_week1.matchups[0].teams[0].points

140.9

In [ ]:
['teamA', 'teamB'] == ['teamB', 'teamA']

False

In [ ]:
frozenset(['teamA', 'teamB']) == frozenset(['teamB', 'teamA'])

True

In [107]:
frozenset([('teamA', 'teamB'), ('teamB', 'teamA'), ('teamA', 'teamB')])

frozenset({('teamA', 'teamB'), ('teamB', 'teamA')})